In [1]:
import urllib
import urllib.request
import http
import http.cookiejar
import lxml
from lxml import etree
import MySQLdb
import time
import re

In [2]:
def get_page_range():
    #查找网页总量
    #get Proxy in https://www.xicidaili.com/nn/
    proxy_url = 'https://www.xicidaili.com/nt/'
    opener = urllib.request.build_opener()
    headers = [('Referer',proxy_url),\
               ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
               ('Cache-Control','max-age=0'),\
               ('Content-Type','application/x-www-form-urlencoded'),\
               ('Origin',proxy_url),\
               ('Connection','keep-alive'),\
               ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
               ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
    opener.addheaders = headers
    request = opener.open(proxy_url)
    request_read = request.read()
    req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
    page_range = req_xml.xpath('//*[@id="body"]/div[3]/a[{}]/text()'.format(len(req_xml.xpath('//*[@id="body"]/div[3]/*'))-4))[0]
    print(page_range)
    opener.close()
    return int(page_range)

In [3]:
def get_proxy_info(proxy_url):
    #获取代理信息
    #proxy_url='https://www.xicidaili.com/nn/3'
    opener = urllib.request.build_opener()
    headers = [('Referer',proxy_url),\
               ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
               ('Cache-Control','max-age=0'),\
               ('Content-Type','application/x-www-form-urlencoded'),\
               ('Origin',proxy_url),\
               ('Connection','keep-alive'),\
               ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
               ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
    opener.addheaders = headers
    #proxy_handler = urllib.request.ProxyHandler({'https':'{}:{}'.format('112.85.165.126',9999),'http':'{}:{}'.format('112.85.165.126',9999)})
    #opener.add_handler(proxy_handler)
    try:
        request = opener.open(proxy_url,timeout=10)
        request_read = request.read()
    except Exception as ex:
        request_read = bytes('Exception:{}'.format(str(ex)),encoding='UTF8')
        print(request_read.decode(encoding='UTF8',errors='ignore'))
        return 0
    req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
    proxy_info_all = []
    for proxy_num in range(2,101):
        proxy_ip=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[2]/text()'.format(proxy_num))[0]))
        proxy_port=(int(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[3]/text()'.format(proxy_num))[0]))
        try:
            proxy_location=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[4]/a/text()'.format(proxy_num))[0]))
        except Exception as ex:
            proxy_location='unknown'
        proxy_anon=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[5]/text()'.format(proxy_num))[0]))
        proxy_type=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[6]/text()'.format(proxy_num))[0]))
        proxy_speed=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[7]/div//@title'.format(proxy_num))[0]))
        proxy_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[8]/div//@title'.format(proxy_num))[0]))
        proxy_life=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[9]/text()'.format(proxy_num))[0]))
        proxy_web_check_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[10]/text()'.format(proxy_num))[0]))
        proxy_info = [proxy_ip,proxy_port,proxy_location,proxy_anon,proxy_type,proxy_speed,proxy_time,proxy_life,proxy_web_check_time]
        proxy_info_all.append(proxy_info)
    print(proxy_info_all)
    opener.close()
    proxy_to_db(proxy_info_all)    

In [4]:
def proxy_to_db(proxy_info_all):
    proxy_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8' )
    proxy_cursor = proxy_db.cursor()
    sql_stmt = "insert into t_proxy_info(proxy_ip,proxy_port,proxy_location,proxy_anon,proxy_type,proxy_speed,proxy_time,proxy_life,proxy_web_check_time)\
            select %s,%s,%s,%s,%s,%s,%s,%s,%s from dual\
            where NOT EXISTS (select 1 from t_proxy_info where proxy_ip = %s and proxy_port = %s)"
    for i in range(len(proxy_info_all)):
        #print(tuple(proxy_info_all[i]+proxy_info_all[i][0:2]))
        proxy_cursor.execute(sql_stmt,tuple(proxy_info_all[i]+proxy_info_all[i][0:2]))
    proxy_db.commit()
    proxy_db.close()

In [5]:
def get_proxy():
    #page_range=get_page_range()
    page_range=1000
    print(page_range)
    for n in range(3,page_range):
        proxy_url = 'https://www.xicidaili.com/nt/{}'.format(n)
        time.sleep(600)
        print(proxy_url)
        get_proxy_info(proxy_url)

In [6]:
def check_proxy():
    proxy_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8')
    proxy_cursor = proxy_db.cursor()
    sql_stmt = "SELECT * FROM NTTask.t_proxy_info"
    proxy_cursor.execute(sql_stmt)
    proxy_list_all = proxy_cursor.fetchall()
    for i in range(len(proxy_list_all)):
        print(proxy_list_all[i])
        check_status = check_proxy_status(proxy_list_all[i])
        print(check_status)
        #插入检查记录
        sql_stmt = 'insert into t_proxy_check_log\
                    values(%s,%s,%s,sysdate())'
        print(list(proxy_list_all[i][0:2])+[check_status[1]])        
        proxy_cursor.execute(sql_stmt,tuple(list(proxy_list_all[i][0:2])+[check_status[1]]))
        
        #更新整体记录表
        if check_status[0] == 1:
            sql_stmt = 'update t_proxy_info a set a.proxy_last_check_time=sysdate(),a.proxy_last_check_status=%s,\
                        a.proxy_all_check_times=ifnull(a.proxy_all_check_times,0)+1,a.proxy_succ_check_times=ifnull(a.proxy_succ_check_times,0)+1\
                        where a.proxy_ip=%s and a.proxy_port=%s'
            proxy_cursor.execute(sql_stmt,tuple([check_status[1]]+list(proxy_list_all[i][0:2])))
        else:
            sql_stmt = 'update t_proxy_info a set a.proxy_last_check_time=sysdate(),a.proxy_last_check_status=%s,\
                        a.proxy_all_check_times=ifnull(a.proxy_all_check_times,0)+1\
                        where a.proxy_ip=%s and a.proxy_port=%s'
            proxy_cursor.execute(sql_stmt,tuple([check_status[1]]+list(proxy_list_all[i][0:2])))
        proxy_db.commit()
    proxy_cursor.close()
    proxy_db.close()

In [7]:
def check_proxy_status(proxy_info):
    if proxy_info[4] == 'HTTPS':
        check_url = 'https://ip.cn/'
    else:
        check_url = 'http://200019.ip138.com/'

    opener = urllib.request.build_opener()
    #添加headers
    headers = [('Referer',check_url),\
               ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
               ('Cache-Control','max-age=0'),\
               ('Content-Type','application/x-www-form-urlencoded'),\
               ('Origin',check_url),\
               ('Connection','keep-alive'),\
               ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
               ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
    opener.addheaders = headers
    proxy_handler = urllib.request.ProxyHandler({'https':'{}:{}'.format(proxy_info[0],proxy_info[1]),'http':'{}:{}'.format(proxy_info[0],proxy_info[1])})
    opener.add_handler(proxy_handler)
    try:
        request = opener.open(check_url,timeout=10)
        request_read = request.read()
        if proxy_info[4] == 'HTTPS':
            req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
            str(req_xml.xpath('//script[1]/text()')[0])
            searchObj = re.search("html\('(.*)'\)",str(req_xml.xpath('//script[1]/text()')[0]), re.M|re.I)
            parse_xml = lxml.etree.HTML(searchObj.group(1))
            check_status = [1,'Success: {}'.format(str(parse_xml.xpath('//div/p/code/text()')))]
        else:
            req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
            check_status = [1,'Success: {}'.format(str(req_xml.xpath('/html/body/p[1]/text()')))]
    except Exception as ex:
        #print('Exception: '+str(ex))
        request_read = bytes('Exception:{}'.format(str(ex)),encoding='UTF8')
        check_status = [0,request_read.decode(encoding='UTF8',errors='ignore')]
    #print(proxy_info[0:2],request_read.decode(encoding='UTF8',errors='ignore'))
    opener.close()
    return check_status

In [ ]:
if __name__ == '__main__':
    get_proxy()
    #check_proxy()

1000
https://www.xicidaili.com/nt/3
[['183.129.207.91', 11056, '浙江杭州', '透明', 'HTTPS', '0.249秒', '0.049秒', '77天', '19-07-06 01:20'], ['106.12.2.99', 3128, 'unknown', '透明', 'HTTPS', '3.528秒', '0.705秒', '12天', '19-07-05 23:20'], ['113.116.245.211', 9797, '广东深圳', '透明', 'HTTP', '0.195秒', '0.039秒', '1分钟', '19-07-05 11:21'], ['119.123.179.163', 9000, '广东深圳', '透明', 'HTTP', '1.356秒', '0.271秒', '1天', '19-07-05 02:20'], ['113.140.1.82', 53281, '陕西西安', '透明', 'HTTPS', '5.633秒', '1.126秒', '313天', '19-07-04 21:00'], ['49.51.155.45', 8081, '上海', '透明', 'HTTPS', '0.733秒', '0.146秒', '1分钟', '19-07-04 12:00'], ['219.131.241.89', 9797, '广东珠海市香洲区', '透明', 'HTTP', '1.659秒', '0.331秒', '1分钟', '19-07-04 11:20'], ['14.20.235.199', 9797, '广东深圳', '透明', 'HTTP', '1.207秒', '0.241秒', '528天', '19-07-04 08:20'], ['175.102.3.98', 8089, '上海', '透明', 'HTTP', '1.753秒', '0.35秒', '694天', '19-07-04 07:00'], ['113.247.252.116', 3128, '湖南长沙', '透明', 'HTTP', '3.164秒', '0.632秒', '92天', '19-07-04 04:20'], ['124.205.143.213', 32612, '北京

https://www.xicidaili.com/nt/4
[['125.123.128.104', 8081, '浙江嘉兴市桐乡', '透明', 'HTTP', '2.557秒', '0.511秒', '1分钟', '19-06-18 16:22'], ['221.7.255.168', 8080, '广西梧州', '透明', 'HTTP', '0.274秒', '0.054秒', '569天', '19-06-18 11:22'], ['14.118.130.213', 8081, '广东江门', '透明', 'HTTPS', '0.207秒', '0.041秒', '306天', '19-06-18 01:00'], ['113.78.254.254', 9000, '广东东莞', '透明', 'HTTP', '0.212秒', '0.042秒', '1分钟', '19-06-17 21:21'], ['101.229.44.238', 9000, '上海', '透明', 'HTTPS', '1.81秒', '0.362秒', '12天', '19-06-17 08:22'], ['112.65.52.28', 9000, '上海', '透明', 'HTTPS', '2.713秒', '0.542秒', '21天', '19-06-17 08:00'], ['125.123.137.124', 9000, '浙江嘉兴', '透明', 'HTTP', '0.201秒', '0.04秒', '73天', '19-06-17 02:20'], ['14.118.130.214', 8081, '广东江门', '透明', 'HTTP', '3.646秒', '0.729秒', '128天', '19-06-17 02:20'], ['116.30.122.12', 9000, '广东深圳', '透明', 'HTTP', '1.437秒', '0.287秒', '1分钟', '19-06-16 03:21'], ['112.95.24.161', 8088, '广东深圳', '透明', 'HTTP', '0.215秒', '0.043秒', '1分钟', '19-06-16 03:21'], ['182.88.27.252', 9797, '广西南宁', '透明', 

https://www.xicidaili.com/nt/5
[['49.82.253.190', 8118, '江苏', '透明', 'HTTPS', '3.161秒', '0.632秒', '5天', '19-06-03 03:00'], ['114.249.115.206', 9000, '北京', '透明', 'HTTP', '4.233秒', '0.846秒', '1分钟', '19-06-03 01:21'], ['117.14.104.71', 53281, '天津', '透明', 'HTTP', '1.41秒', '0.282秒', '1分钟', '19-06-03 01:20'], ['119.48.190.218', 9999, '吉林长春', '透明', 'HTTP', '0.136秒', '0.027秒', '1分钟', '19-06-03 00:20'], ['114.249.117.178', 9000, '北京', '透明', 'HTTP', '2.509秒', '0.501秒', '1分钟', '19-06-02 18:20'], ['112.95.188.3', 9797, '广东深圳', '透明', 'HTTP', '5.459秒', '1.091秒', '1分钟', '19-06-02 16:20'], ['121.232.199.179', 9000, '江苏镇江', '透明', 'HTTPS', '0.137秒', '0.027秒', '1分钟', '19-06-02 10:00'], ['120.76.62.125', 3128, 'unknown', '透明', 'HTTP', '0.142秒', '0.028秒', '8天', '19-06-02 10:00'], ['202.98.76.166', 8118, '云南昆明', '透明', 'HTTP', '0.387秒', '0.077秒', '1分钟', '19-06-02 03:20'], ['222.74.61.98', 53281, '内蒙古包头', '透明', 'HTTP', '0.206秒', '0.041秒', '292天', '19-06-02 02:20'], ['121.232.199.187', 9000, '江苏镇江', '透明', 'HTTP

https://www.xicidaili.com/nt/6
[['120.79.176.70', 3128, 'unknown', '透明', 'HTTPS', '2.88秒', '0.576秒', '1分钟', '19-05-24 23:00'], ['202.112.237.102', 3128, '北京', '透明', 'HTTPS', '0.111秒', '0.022秒', '237天', '19-05-24 21:00'], ['218.94.141.59', 21070, '江苏南京', '透明', 'HTTPS', '3.088秒', '0.617秒', '16小时', '19-05-24 19:00'], ['218.94.141.58', 21070, '江苏南京', '透明', 'HTTP', '5.749秒', '1.149秒', '19小时', '19-05-24 19:00'], ['113.116.57.10', 9000, '广东深圳', '透明', 'HTTP', '6.524秒', '1.304秒', '1分钟', '19-05-24 16:21'], ['117.90.252.37', 9000, '江苏镇江', '透明', 'HTTP', '0.518秒', '0.103秒', '1分钟', '19-05-24 11:20'], ['119.139.196.29', 3128, '广东深圳', '透明', 'HTTP', '0.168秒', '0.033秒', '1分钟', '19-05-24 08:21'], ['222.186.45.145', 57273, '江苏镇江', '透明', 'HTTP', '0.591秒', '0.118秒', '426天', '19-05-24 08:00'], ['27.46.21.83', 8888, '广东深圳', '透明', 'HTTP', '0.205秒', '0.041秒', '610天', '19-05-24 03:21'], ['117.90.6.244', 9000, '江苏镇江', '透明', 'HTTP', '0.204秒', '0.04秒', '1分钟', '19-05-24 02:20'], ['59.38.62.2', 9797, '广东珠海', '透明', 'H

https://www.xicidaili.com/nt/7
[['163.125.16.225', 8888, '广东深圳', '透明', 'HTTPS', '1.231秒', '0.246秒', '1分钟', '19-05-17 01:00'], ['114.101.41.195', 65309, '安徽蚌埠', '透明', 'HTTPS', '2.679秒', '0.535秒', '1分钟', '19-05-17 01:00'], ['121.232.199.59', 9000, '江苏镇江', '透明', 'HTTP', '0.139秒', '0.027秒', '1分钟', '19-05-17 00:20'], ['123.13.247.95', 9999, '河南许昌', '透明', 'HTTP', '4.133秒', '0.826秒', '13天', '19-05-16 21:20'], ['222.186.45.160', 65301, '江苏镇江', '透明', 'HTTPS', '0.374秒', '0.074秒', '143天', '19-05-16 18:00'], ['163.125.68.135', 8888, '广东深圳', '透明', 'HTTP', '3.742秒', '0.748秒', '1分钟', '19-05-16 09:20'], ['14.115.104.97', 808, '广东珠海', '透明', 'HTTP', '0.172秒', '0.034秒', '1分钟', '19-05-16 07:20'], ['14.115.106.12', 808, '广东珠海', '透明', 'HTTP', '0.219秒', '0.043秒', '1分钟', '19-05-16 07:20'], ['115.225.123.26', 8118, '浙江绍兴', '透明', 'HTTPS', '0.191秒', '0.038秒', '1分钟', '19-05-16 05:00'], ['121.238.82.201', 8118, '江苏南通', '透明', 'HTTP', '0.152秒', '0.03秒', '7小时', '19-05-16 04:20'], ['117.80.203.174', 8118, '江苏苏州', '透明'

...

https://www.xicidaili.com/nt/12
[['113.109.29.25', 9797, '广东广州', '透明', 'HTTP', '0.195秒', '0.039秒', '1分钟', '19-03-13 19:20'], ['125.123.22.216', 9000, '浙江嘉兴', '透明', 'HTTP', '0.199秒', '0.039秒', '1分钟', '19-03-13 17:20'], ['125.123.123.64', 9000, '浙江嘉兴市桐乡', '透明', 'HTTP', '3.699秒', '0.739秒', '1分钟', '19-03-13 17:20'], ['27.37.47.244', 9000, '广东东莞', '透明', 'HTTP', '1.345秒', '0.269秒', '1分钟', '19-03-13 16:21'], ['125.123.69.35', 9000, '浙江嘉兴', '透明', 'HTTP', '1.361秒', '0.272秒', '1分钟', '19-03-13 15:20'], ['125.123.122.17', 9000, '浙江嘉兴市桐乡', '透明', 'HTTPS', '2.94秒', '0.588秒', '1分钟', '19-03-13 14:00'], ['113.116.144.188', 9000, '广东深圳', '透明', 'HTTPS', '1.802秒', '0.36秒', '1分钟', '19-03-13 14:00'], ['112.95.22.182', 8888, '广东深圳', '透明', 'HTTPS', '5.254秒', '1.05秒', '329天', '19-03-13 12:20'], ['14.115.105.78', 808, '广东珠海', '透明', 'HTTP', '1.655秒', '0.331秒', '1分钟', '19-03-13 12:20'], ['119.57.108.73', 53281, '北京', '透明', 'HTTPS', '1.399秒', '0.279秒', '588天', '19-03-13 12:00'], ['202.4.153.40', 808, '北京', '透明', 'H

https://www.xicidaili.com/nt/13
[['49.4.12.248', 3128, '北京', '透明', 'HTTPS', '0.029秒', '0.005秒', '1分钟', '19-03-06 08:00'], ['118.81.45.248', 9797, '山西太原', '透明', 'HTTP', '0.068秒', '0.013秒', '1分钟', '19-03-06 03:20'], ['125.39.25.249', 8082, '天津', '透明', 'HTTPS', '0.039秒', '0.007秒', '1分钟', '19-03-06 03:00'], ['125.123.136.74', 9000, '浙江嘉兴', '透明', 'HTTP', '7.675秒', '1.535秒', '213天', '19-03-05 23:00'], ['119.123.176.28', 9000, '广东深圳', '透明', 'HTTP', '1.892秒', '0.378秒', '428天', '19-03-05 22:21'], ['125.123.143.18', 9000, '浙江嘉兴', '透明', 'HTTPS', '7.68秒', '1.536秒', '156天', '19-03-05 21:00'], ['14.20.235.100', 808, '广东深圳', '透明', 'HTTP', '4.584秒', '0.916秒', '2天', '19-03-05 17:00'], ['162.105.95.73', 8888, '北京', '透明', 'HTTP', '1.466秒', '0.293秒', '1分钟', '19-03-05 15:20'], ['14.20.235.190', 9797, '广东深圳', '透明', 'HTTPS', '3.962秒', '0.792秒', '3天', '19-03-05 15:00'], ['163.125.65.170', 9797, '广东深圳', '透明', 'HTTP', '0.247秒', '0.049秒', '1分钟', '19-03-05 09:20'], ['163.125.65.179', 9797, '广东深圳', '透明', 'HTTPS', 

https://www.xicidaili.com/nt/14
[['116.62.199.223', 8118, '黑龙江', '透明', 'HTTP', '1.124秒', '0.224秒', '1分钟', '19-02-12 02:21'], ['163.125.157.162', 8888, '广东深圳', '透明', 'HTTPS', '1.938秒', '0.387秒', '1分钟', '19-02-11 10:00'], ['112.95.16.223', 8088, '广东深圳', '透明', 'HTTP', '1.913秒', '0.382秒', '1分钟', '19-02-11 09:20'], ['182.88.148.148', 9797, '广西南宁', '透明', 'HTTPS', '1.2秒', '0.24秒', '1分钟', '19-02-11 09:00'], ['115.171.203.201', 9000, '北京', '透明', 'HTTP', '3.616秒', '0.723秒', '1分钟', '19-02-10 22:21'], ['219.159.38.201', 56210, '内蒙古呼和浩特', '透明', 'HTTP', '3.621秒', '0.724秒', '55天', '19-02-10 11:20'], ['118.81.45.146', 9797, '山西太原', '透明', 'HTTPS', '3.066秒', '0.613秒', '1分钟', '19-02-10 09:00'], ['1.192.242.217', 9999, '河南郑州', '透明', 'HTTPS', '0.121秒', '0.024秒', '382天', '19-02-09 14:00'], ['125.123.143.251', 9000, '浙江嘉兴', '透明', 'HTTPS', '0.152秒', '0.03秒', '197天', '19-02-09 09:00'], ['182.88.247.177', 9797, '广西南宁', '透明', 'HTTP', '1.202秒', '0.24秒', '1分钟', '19-02-08 20:21'], ['117.62.61.180', 8118, '江苏', '透明'

https://www.xicidaili.com/nt/15
[['125.123.21.62', 9000, '浙江嘉兴', '透明', 'HTTPS', '5.519秒', '1.103秒', '1分钟', '19-01-09 14:00'], ['116.62.4.184', 8118, '广东佛山', '透明', 'HTTPS', '0.277秒', '0.055秒', '42天', '19-01-09 11:00'], ['222.52.142.242', 8080, '广西桂林', '透明', 'HTTP', '0.671秒', '0.134秒', '1001天', '19-01-09 10:20'], ['119.123.175.14', 9797, '广东深圳', '透明', 'HTTP', '2.094秒', '0.418秒', '1分钟', '19-01-09 09:20'], ['125.123.20.37', 9000, '浙江嘉兴', '透明', 'HTTP', '2.774秒', '0.554秒', '1分钟', '19-01-09 03:20'], ['112.95.205.173', 8888, '广东深圳', '透明', 'HTTPS', '0.224秒', '0.044秒', '1分钟', '19-01-09 00:00'], ['123.117.36.50', 9000, '北京', '透明', 'HTTP', '7.274秒', '1.454秒', '1分钟', '19-01-08 13:20'], ['139.159.246.39', 8118, 'unknown', '透明', 'HTTPS', '1.123秒', '0.224秒', '39天', '19-01-08 12:20'], ['112.67.187.156', 9797, '海南海口', '透明', 'HTTP', '0.277秒', '0.055秒', '1分钟', '19-01-08 03:20'], ['183.33.130.46', 8118, '广东珠海', '透明', 'HTTPS', '0.174秒', '0.034秒', '1分钟', '19-01-07 22:00'], ['119.51.89.18', 1080, '吉林长春', '透明'

https://www.xicidaili.com/nt/16
[['114.249.112.55', 9000, '北京', '透明', 'HTTP', '5.837秒', '1.167秒', '1分钟', '18-12-29 10:45'], ['114.249.114.2', 9000, '北京', '透明', 'HTTPS', '5.133秒', '1.026秒', '3天', '18-12-29 09:11'], ['124.205.155.151', 9090, '北京', '透明', 'HTTP', '2.076秒', '0.415秒', '435天', '18-12-29 09:01'], ['113.128.140.98', 8118, '山东济南', '透明', 'HTTPS', '0.089秒', '0.017秒', '74天', '18-12-29 08:45'], ['27.46.21.206', 8888, '广东深圳', '透明', 'HTTP', '1.522秒', '0.304秒', '375天', '18-12-29 08:30'], ['222.218.140.92', 8118, '广西河池', '透明', 'HTTP', '0.228秒', '0.045秒', '1分钟', '18-12-29 07:15'], ['119.123.179.150', 9000, '广东深圳', '透明', 'HTTPS', '0.204秒', '0.04秒', '3天', '18-12-29 05:00'], ['61.160.233.112', 10000, '江苏常州', '透明', 'HTTPS', '0.15秒', '0.03秒', '7天', '18-12-29 01:45'], ['210.77.2.127', 1080, '北京', '透明', 'HTTPS', '0.745秒', '0.149秒', '1分钟', '18-12-29 00:22'], ['180.160.30.11', 8123, '上海', '透明', 'HTTPS', '3.981秒', '0.796秒', '1分钟', '18-12-29 00:11'], ['113.128.140.234', 8118, '山东济南', '透明', 'HTTPS',

In [ ]:
while True:
    check_proxy()

('182.34.20.220', 9999, '山东烟台', '高匿', 'HTTP', '0.109秒', '0.021秒', '1分钟', '19-07-26 10:20', datetime.datetime(2019, 7, 26, 13, 6, 18), 'Exception:<urlopen error timed out>', 2, None)
[0, 'Exception:<urlopen error timed out>']
['182.34.20.220', 9999, 'Exception:<urlopen error timed out>']
('1.197.204.7', 9999, '河南济源', '高匿', 'HTTP', '0.089秒', '0.017秒', '1分钟', '19-07-26 10:20', datetime.datetime(2019, 7, 26, 13, 6, 21), 'Exception:<urlopen error [Errno 111] Connection refused>', 2, None)
[0, 'Exception:<urlopen error timed out>']
['1.197.204.7', 9999, 'Exception:<urlopen error timed out>']
('113.121.22.254', 9999, '山东', '高匿', 'HTTP', '0.268秒', '0.053秒', '5天', '19-07-26 10:20', datetime.datetime(2019, 7, 26, 13, 6, 21), "Success: ['\\r\\n您的IP地址是：[113.121.22.254] 来自：山东省烟台市 电信\\r\\n']", 2, 1)
[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['113.121.22.254', 9999, 'Exception:<urlopen error [Errno 111] Connection refused>']
('120.83.105.25', 9999, '广东揭阳', '高匿', 'HTTP', '0.189秒'

[0, 'Exception:<urlopen error timed out>']
['115.204.37.62', 53281, 'Exception:<urlopen error timed out>']
('49.86.182.232', 9999, '江苏', '高匿', 'HTTP', '0.145秒', '0.029秒', '1分钟', '19-07-26 07:20', datetime.datetime(2019, 7, 26, 13, 9, 36), 'Exception:<urlopen error timed out>', 2, None)
[0, 'Exception:<urlopen error timed out>']
['49.86.182.232', 9999, 'Exception:<urlopen error timed out>']
('115.206.253.190', 8118, '浙江杭州', '高匿', 'HTTP', '0.154秒', '0.03秒', '1分钟', '19-07-26 07:20', datetime.datetime(2019, 7, 26, 13, 9, 36), "Success: ['\\r\\n您的IP地址是：[115.206.253.190] 来自：浙江省杭州市 电信\\r\\n']", 2, 1)
[1, "Success: ['\\r\\n您的IP地址是：[115.206.253.190] 来自：浙江省杭州市 电信\\r\\n']"]
['115.206.253.190', 8118, "Success: ['\\r\\n您的IP地址是：[115.206.253.190] 来自：浙江省杭州市 电信\\r\\n']"]
('27.43.189.46', 9999, '广东梅州', '高匿', 'HTTPS', '0.189秒', '0.037秒', '1分钟', '19-07-26 07:00', datetime.datetime(2019, 7, 26, 13, 9, 38), 'Exception:<urlopen error [Errno 111] Connection refused>', 2, None)
[0, 'Exception:<urlopen error [E

[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['113.128.121.81', 9999, 'Exception:<urlopen error [Errno 111] Connection refused>']
('1.198.73.89', 9999, '河南', '高匿', 'HTTPS', '0.102秒', '0.02秒', '1分钟', '19-07-26 04:00', datetime.datetime(2019, 7, 26, 13, 11, 39), 'Exception:<urlopen error timed out>', 2, None)
[0, 'Exception:<urlopen error timed out>']
['1.198.73.89', 9999, 'Exception:<urlopen error timed out>']
('163.204.243.187', 9999, '广东', '高匿', 'HTTP', '1.21秒', '0.242秒', '70天', '19-07-26 04:00', datetime.datetime(2019, 7, 26, 13, 11, 49), 'Exception:<urlopen error timed out>', 2, None)
[0, 'Exception:<urlopen error timed out>']
['163.204.243.187', 9999, 'Exception:<urlopen error timed out>']
('124.93.201.59', 59618, '辽宁大连', '高匿', 'HTTPS', '4.658秒', '0.931秒', '260天', '19-07-26 04:00', datetime.datetime(2019, 7, 26, 13, 11, 51), 'Exception:<urlopen error [Errno 111] Connection refused>', 2, None)
[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['124.9

In [ ]:
proxy_info = ('221.178.146.113', 61399, '广东深圳', '高匿', 'HTTPS', '2.524秒', '0.504秒', '1分钟', '19-06-25 13:21', None, None)
if proxy_info[4] == 'HTTPS':
    check_url = 'https://ip.cn/'
else:
    check_url = 'http://200019.ip138.com/'
    
opener = urllib.request.build_opener()
#添加headers
headers = [('Referer',check_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',check_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers
proxy_handler = urllib.request.ProxyHandler({'https':'{}:{}'.format(proxy_info[0],proxy_info[1]),'http':'{}:{}'.format(proxy_info[0],proxy_info[1])})
opener.add_handler(proxy_handler)
try:
    request = opener.open(check_url,timeout=10)
    request_read = request.read()
    req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
    str(req_xml.xpath('//script[1]/text()')[0])
    searchObj = re.search("html\('(.*)'\)",str(req_xml.xpath('//script[1]/text()')[0]), re.M|re.I)
    print(searchObj.group(1))
    parse_xml = lxml.etree.HTML(searchObj.group(1))
    print('Success: {}'.format(str(parse_xml.xpath('//div/p/code/text()'))))
except Exception as ex:
    #print('Exception: '+str(ex))
    request_read = bytes('Exception:{}'.format(str(ex)),encoding='UTF8')
print(proxy_info[0:2],request_read.decode(encoding='UTF8',errors='ignore'))
opener.close()

In [ ]:
print(str(req_xml.xpath('//script[1]/text()')[0]))

In [ ]:
check_proxy()

In [ ]:
opener = urllib.request.build_opener()
proxy_url='https://www.xicidaili.com/nn/1'
headers = [('Referer',proxy_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',proxy_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers
print(proxy_url)
request = opener.open(proxy_url)
request_read = request.read()
#print(request_read.decode(encoding='UTF8',errors='ignore'))
req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))

In [ ]:
#print(str(req_xml.xpath('//*[@id="ip_list"]/tr[101]/td[1]/text()')[0]))
for proxy_num in range(2,101):
    proxy_ip=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[2]/text()'.format(proxy_num))[0]))
    proxy_port=(int(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[3]/text()'.format(proxy_num))[0]))
    try:
        proxy_location=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[4]/a/text()'.format(proxy_num))[0]))
    except Exception as ex:
        proxy_location='unknown'
    proxy_anon=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[5]/text()'.format(proxy_num))[0]))
    proxy_type=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[6]/text()'.format(proxy_num))[0]))
    proxy_speed=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[7]/div//@title'.format(proxy_num))[0]))
    proxy_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[8]/div//@title'.format(proxy_num))[0]))
    proxy_life=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[9]/text()'.format(proxy_num))[0]))
    proxy_web_check_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[10]/text()'.format(proxy_num))[0]))
    proxy_info = [proxy_ip,proxy_port,proxy_location,proxy_anon,proxy_type,proxy_speed,proxy_time,proxy_life,proxy_web_check_time]
    print(proxy_info)

In [ ]:
test_url = 'http://200019.ip138.com/'
test_https = 'https://ip.cn/'
opener = urllib.request.build_opener()
#添加headers
headers = [('Referer',proxy_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',proxy_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers

#添加cookie
#cookie_filename = 'cookie_xicidaili.txt'
#cookies = http.cookiejar.CookieJar()
#cookies = http.cookiejar.MozillaCookieJar(cookie_filename)
#cookies.load(ignore_discard=True, ignore_expires=True)
#cookie_handler = urllib.request.HTTPCookieProcessor(cookies)
#opener.add_handler(cookie_handler)
#添加代理
#proxy_handler = urllib.request.ProxyHandler({'https':'221.178.146.113:61399','http':'221.178.146.113:61399'})
proxy_handler = urllib.request.ProxyHandler({'https':'222.89.32.154:9999','http':'222.89.32.154:9999'})
opener.add_handler(proxy_handler)

#添加PostData
#data = {"jumpurl":"http%3A%2F%2Fwww.itokoo.com%2Findex.php","step":"2","ajax":"1","pwuser":"Flashield","pwpwd":"Zb122525","lgt":"0"}
#post_data = urllib.parse.urlencode(data).encode(encoding='UTF8')


In [ ]:
try:
    request = opener.open(test_https,timeout=10)
    request_read = request.read()
    req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
    req_xml.xpath('//*[@id="result"]/div/p[1]/code/text()')
    print(req_xml.xpath('//*[@id="result"]/div/p[1]/code/text()')[0])
except Exception as ex:
        print('Exception:{}'.format(str(ex)))

In [ ]:
request = opener.open(proxy_url,ti)
request_read = request.read()
req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
proxy_ip = req_xml.xpath('//*[@id="ip_list"]/tr[100]/td[2]/text()')[0]
proxy_port = req_xml.xpath('//*[@id="ip_list"]/tr[100]/td[3]/text()')[0]
proxy_type = req_xml.xpath('//*[@id="ip_list"]/tr[100]/td[6]/text()')[0]
print(proxy_ip,proxy_port,proxy_type)
proxy_handler = urllib.request.ProxyHandler({proxy_type:'{}:{}'.format(proxy_ip, proxy_port)})
opener1 = urllib.request.build_opener()
opener1.add_handler(proxy_handler)

In [ ]:
request = opener1.open(test_url)
request_read = request.read()
request_read.decode(encoding='UTF8',errors='ignore')

In [ ]:
with open('Proxy.html','w') as f:
    f.write(request_read.decode(encoding='GB2312',errors='ignore'))

In [ ]:
req_xml.xpath('//*[@id="result"]/div/p[1]/code/text()')

In [ ]:
req_xml.xpath('//*[@id="body"]/div[2]/a[{}]/text()'.format(len(req_xml.xpath('//*[@id="body"]/div[2]/*'))-4))

In [ ]:
proxy_url='https://www.xicidaili.com/nn/6'
opener = urllib.request.build_opener()
headers = [('Referer',proxy_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',proxy_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers
proxy_handler = urllib.request.ProxyHandler({'https':'{}:{}'.format('163.204.246.41',9999),'http':'{}:{}'.format('163.204.246.41',9999)})
opener.add_handler(proxy_handler)
try:
    request = opener.open(proxy_url,timeout=10)
    request_read = request.read()
except Exception as ex:
    request_read = bytes('Exception:{}'.format(str(ex)),encoding='UTF8')
print(request_read.decode(encoding='UTF8',errors='ignore'))
#return 0
req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
proxy_info_all = []
for proxy_num in range(2,101):
    proxy_ip=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[2]/text()'.format(proxy_num))[0]))
    proxy_port=(int(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[3]/text()'.format(proxy_num))[0]))
    try:
        proxy_location=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[4]/a/text()'.format(proxy_num))[0]))
    except Exception as ex:
        proxy_location='unknown'
    proxy_anon=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[5]/text()'.format(proxy_num))[0]))
    proxy_type=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[6]/text()'.format(proxy_num))[0]))
    proxy_speed=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[7]/div//@title'.format(proxy_num))[0]))
    proxy_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[8]/div//@title'.format(proxy_num))[0]))
    proxy_life=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[9]/text()'.format(proxy_num))[0]))
    proxy_web_check_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[10]/text()'.format(proxy_num))[0]))
    proxy_info = [proxy_ip,proxy_port,proxy_location,proxy_anon,proxy_type,proxy_speed,proxy_time,proxy_life,proxy_web_check_time]
    proxy_info_all.append(proxy_info)
print(proxy_info_all)
opener.close()
proxy_to_db(proxy_info_all)  

In [8]:
proxy_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8')

In [9]:
proxy_db.close()